# Предобработка данных

## Импорт библиотек

In [47]:
import pandas as pd
import os

import pytesseract
from PIL import Image

import cv2

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Загрузка и осмотр данных

In [41]:
shape_counter = 0
nan_counter = 0
columns = 10
df_list = []

for filename in os.listdir('405/'):
    if 'csv' in filename:
        path = '405/' + filename
        df = pd.read_csv(path)
        print('\n', filename)
        print(df.head())
        df_list.append(df)
        if df.shape[1] != columns:
            shape_counter += 1
        if df.isna().sum().sum() != 0:
            nan_counter += 1
print('\n', 'Число файлов где количество столбцов отличается от 10---', shape_counter)
print('\n', 'Число файлов с пропусками---', nan_counter)


 141899.csv
       ID  ID пользователя    Класс крови Дата донации  \
0   87330           141899  Цельная кровь   21.06.2019   
1  213527           141899         Плазма   29.08.2019   
2  213528           141899         Плазма   30.09.2019   
3  213529           141899         Плазма   01.11.2019   
4  213530           141899         Плазма   02.12.2019   

  Дата добавления донации   Тип донации                          Регион  \
0              18.11.2020  Безвозмездно  Россия, Татарстан, Альметьевск   
1              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
2              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
3              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
4              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   

   Место стадчи Статус донации Есть справка  
0           703        Принята           Да  
1           703        Принята           Да  
2           703        Принята           Да  
3  

**Чтож, неплохо. все таблицы имеют одинаковое число столбцов, пропусков нет. Жить можно**

## Добавляем город и убираем лишние столбцы

In [42]:
def region_to_city(st):
    return st.split(', ')[-1]

In [43]:
trash_columns = ['ID', 'ID пользователя', 'Дата добавления донации', 'Место стадчи', 'Статус донации', 'Есть справка', 'Регион']

for i in range(len(df_list)):
    df_list[i]['Город'] = df_list[i]['Регион'].apply(region_to_city)
    df_list[i] = df_list[i].drop(trash_columns, axis=1)

In [44]:
df_list[5].head()

,Класс крови,Дата донации,Тип донации,Город
0,Цельная кровь,03.07.2009,Безвозмездно,Ульяновск
1,Цельная кровь,16.09.2009,Безвозмездно,Ульяновск
2,Цельная кровь,16.03.2011,Безвозмездно,Ульяновск
3,Цельная кровь,01.08.2011,Безвозмездно,Ульяновск
4,Цельная кровь,24.11.2011,Безвозмездно,Ульяновск


# Первые попытки что-то распознать

In [50]:
image = cv2.imread('405/141899 .jpg')

preprocess = "thresh"
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if preprocess == "thresh":
    gray = cv2.threshold(gray, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
elif preprocess == "blur":
    gray = cv2.medianBlur(gray, 3)
    
filename = "{}.png".format(os.getpid())
cv2.imwrite(filename, gray)



string = pytesseract.image_to_string(gray, lang='rus')

print(string)

Виа.
риа.

В

2

21.08.2019

Ба

95.58.2020 | пл ин

вл

2908-2018

гул (68

25082020 | моба)

СНЕ

30.09-2019

ур 65)

2850200 | пилив®

05202 пи

СЕ

полу ба7

1637.202% [ вмз)

05062022 оля

[021212076]

2762

23032021 | плид (и)

ЧТОТО РМАТ 80

20.2020)

лоу 68)

ово [ов

[30а 52 пни

С

27.0.2020)

46)

СЕТИ

44022085 ил

1#355.2920]

"Уд 169}

СЕИСО

р

ВВ

11.06.2620

СС]

19082021 | пд 1687

ет

19.7.2626]

лу)

ва [аа | м6

“)

ры

регланы тот: бете ттт




In [51]:
image_1 = Image.open('405/141899 .jpg')
string = pytesseract.image_to_string(image_1, lang='rus')

print(string)

"Ответственное лицо 0 Ик

"Отв вить кров (плазмы, «лоток «ров

дата: д кт»

т
[206 20т9 С

[2908019 25082025 | чб)

30092018] пуд (68) 26102020 | па (ва) |

6192919] по (60) 61.2020 | чмо)

[0292-2019] плул69) 3092021 | пав) | 80

о идею) | 600 [0202 | лав [6 | ава)
27.03.2026] пуд (69) 25082021 | пд (68) |600 в)
13.05.2020] пл/д (68) 19.07.2021 | пм/д(6в)

о62о2О[ има (6) | оао

0072020 | плйд (68) 14192021 | пуд (69) | 60

енатю вАНСТ16 нм 25


